![Wolfe](https://s3.amazonaws.com/lquant-images/wolfe_luo.jpg)

# Overview of Python Risk Model API
**pyqes.micsvc**:

Python package to utilize the Luo QES API service functions. The API can be used to build historical risk models. It provides a convenient way to build a risk model using standard or customized universes. User can select 

1. Risk Universe
2. List of factors
3. Parameters for estimation

The risk model parameters are driven from risk model templates. We provide standard risk model templates that can be copied/edited/saved as user specific templates. 



## Requirements and Presteps

1. Copy pyqes [python file]( https://github.com/wolferesearch/docs/tree/master/micro-services/api/python/pyqes) from github to your local directory from Github. 
2. Ensure you have [Pandas](https://pandas.pydata.org/) and [requests](https://pypi.org/project/requests/) package in your python kernel. 

## Authentication and Connection

The API is protected using Username and Password. In case you have not received it, please [email](mailto:luo.qes@wolferesearch.com) to apply for API account. 

The connection object is the gateway to accessing the API. It allows you to access the catalog, portfolios, templates, risk models etc. 

In [1]:
# The micsvc folder should be in your working directory
from pyqes import micsvc

user = '********'

# create a connection object to interact with the server
connection = micsvc.Connection(username = user, password = '*******')

### Connection Class

Provides convenients way to interface with the Cataglog/Risk/Template RESTful API. The first step is to browse the catalog that provides access to Universes and Factors. 

In [2]:
# Catalog instance allows you browse different entities in the database...
catalog = connection.get_catalog()

### Catalog

* <ins>**Universe**</ins> : Time varying collection of stocks. We have wide array of universe available. Additional universes can be added to the collection on client's request. 


* <ins>**Factor**</ins>: List of available factors (or features, e.g., Earnings Yield, ROE, ..) for stocks. The factors form the basis for risk model and capturing the systematic risk. The factors can be added to the risk model template to enhance the model. While adding more factor can better capture the systematic risk, caution should be exercised in order to avoid overfitting. Also, it is advisable to add correlated factors as they can cause multi-collinearity. 


* <ins>**Meta Factor**</ins>: List of meta data available (e.g., Market Cap, Ticker, ..) for stocks. This is for reference and mapping purpose. 


* <ins>**Portfolio**</ins>: User can upload portfolio to the database. Portfolios are saved in users own space. The purpose of the portfolio is to be used as a custom universe for risk model and other requests. 


* <ins>**Template**</ins>: A template is stored parameter set for the risk model requests. The two main purposes of templates are (1) Simplifying the API, and (2) Reuse parameters across different requests. 

### Factors

We have a wide array of factors available capturing fundamental and technical themes of the stocks. The factors are based on years of extensive alpha/risk research by the QES team. The catolog provides an easy way to list down available factors. In case you are interested in additional factors, please [email](mailto:luo.qes@wolferesearch.com) desk. 

In [3]:
catalog.get_factors().head()

,CATEGORY,DESCRIPTION,ID,SUBCATEGORY
0,Value,"Book-to-market, FY1",BOOKP_FY1,Book
1,Value,"Price-to-EPS, LTM, diluted",PE_LTM_D,Earnings
2,Value,"Operating cash flow yield, FY1 (= Last 12 mont...",CFOYLD_FY1,Cash flow
3,Value,"Earnings yield (LTM, basic) x 5Y Exp Growth",EPSYLD_GRO,GARP
4,Value,"Earnings yield, LTM, operating",EPSYLD_LTM_O,Earnings


### Universe

Collection of standardized universe that can be used as risk universe. Note that custom portfolios can also be used as risk universe. The ID colun in the table below uniquely identifies the universe. 

In [4]:
catalog.get_universe().head()

,COUNTRY,DESCRIPTION,ID,NAME,SECTOR
0,US,Universe stocks from US based on Market Capita...,US_1,US Broad Market Index,All
1,US,Universe provides constituents for S&P 500,SP500,QES 500,All
2,US,Universe provides constituents for S&P 1500,SP1500,QES 1500,All
3,US,Russell 2000 Index,CIQ_INDEX_2667223,Russell 2000 Index,All
4,US,Russell 1000 Index,CIQ_INDEX_2668794,Russell 1000 Index,All


### Portfolio

Similar to universe but user can upload it along with custom factors. A single portfolio added to the database can provide a risk universe as well as custom factors. Note that any columns other than date/id are treated as custom factors). For more details on format of the CSV file, please see our [Github link](https://github.com/wolferesearch/docs/blob/master/portfolio.md). You can also download a sample file using this [link](https://raw.githubusercontent.com/wolferesearch/docs/master/sample/LongFormatPort.csv). 

In [5]:
catalog.get_portfolios().head(3)


,ID,UPLOADEDBY,UPLOADEDTIME
0,HJAIN_MY_CUSTOM_PORTFOLIO,hjain,2023-02-01T00:00:00.000Z
1,Custom_Port,hjain,2019-02-13T00:00:00.000Z
2,A1A2,hjain,2019-02-13T00:00:00.000Z


### Uploading a portfolio

A simple CSV file can be uploaded a custom list. We support Ticker (US Only), SEDOL, CUSIP, BBTICKER as valid identifiers. 


In [6]:
# Always prefix the user with the username to avoid conflict
user = 'hjain'
# You can also download a sample file using this https://raw.githubusercontent.com/wolferesearch/docs/master/sample/LongFormatPort.csv
portfolio_id = '{}_MY_CUSTOM_PORTFOLIO'.format(user.upper())
portfolio = connection.upload_portfolio(id = portfolio_id, filename = '../../../sample/LongFormatPort.csv')

In [8]:
# Check if the portfolio is uploaded successfully
portfolio.text

'file uploaded successfully'

In [9]:
catalog.get_portfolios().head(3)

,ID,UPLOADEDBY,UPLOADEDTIME
0,HJAIN_MY_CUSTOM_PORTFOLIO,hjain,2023-02-01T00:00:00.000Z
1,Custom_Port,hjain,2019-02-13T00:00:00.000Z
2,A1A2,hjain,2019-02-13T00:00:00.000Z


<span style="color:red">*Note that a portfolio to be used a risk model universe, it is important to have wide and deep coverage. The example above is a very simple portfolio. It is recommended to provide portfolio with at least one month frequency* and have at least 10-20 stocks in each of the GICS industry group</span>

### Templates

Templates are stored parameters that can be reused. Templates are stored as Json file in the database. Users can take an existing template and modify it and save it back in the database. The template will be stored under individual users space and only visible to the account you are using to connect. 

In [11]:
## Get all templates using the catalog class
catalog.get_templates().head()

,CONTENT,DESCRIPTION,MODIFIED_DATE,NAME,OWNER,TYPE
0,"{'meta': [{'name': 'Sedol', 'mnemonic': 'SEDOL...",Model uses default set of factors and grouping...,2019-01-18T16:34:39.000000,default,admin,Risk-Model
1,"{'global': {'currency': 'EUR'}, 'meta': [{'nam...",Model is build for Euro with default currency ...,2023-01-04T15:42:10.000000,default-euro,admin,Risk-Model
2,"{'meta': [{'name': 'Sedol', 'mnemonic': 'SEDOL...",default3,2019-01-29T23:28:00.000000,default3,hjain,Risk-Model
3,"{'meta': [{'name': 'Sedol', 'mnemonic': 'SEDOL...",default-hj1,2019-01-17T22:51:04.000000,default-hj1,hjain,Risk-Model
4,"{'meta': [{'name': 'Sedol', 'mnemonic': 'SEDOL...",default37,2019-01-29T23:28:19.000000,default37,hjain,Risk-Model


In [12]:
default_risk_template = connection.get_template(name = 'default', type_ = 'Risk-Model')

### Inspecting the Template

In [13]:
default_risk_template.json

{'__name__': 'default',
 '__type__': 'ltool.risk.RiskModelBuilder',
 'covArgs': {'cov.period': 252,
  'expWeight': 'true',
  'interval': 3,
  'var.period': 84},
 'factors': [{'mnemonic': 'EPSYLD_LTM_B', 'name': 'Earnings Yld'},
  {'mnemonic': 'GR_INTR_EPS', 'name': 'EPS Growth (YoY)'},
  {'mnemonic': 'RTN_12M1M', 'name': 'Momentum (12M-1M)'},
  {'mnemonic': 'ES_EPS_NTM_R3M', 'name': 'Revision'},
  {'mnemonic': 'ROE', 'name': 'Profitability'},
  {'mnemonic': 'REAL_VOL', 'name': 'Volatility'},
  {'mnemonic': 'MKTCAP_M_USD', 'name': 'Size (Mkt. Cap)'},
  {'mnemonic': 'DIVYLD_TRL', 'name': 'Div Yield'},
  {'mnemonic': 'BOOKP', 'name': 'Book to Market'}],
 'meta': [{'mnemonic': 'SEDOL', 'name': 'Sedol'},
  {'mnemonic': 'TICKER', 'name': 'Ticker'},
  {'mnemonic': 'COMPANYNAME', 'name': 'Company Name'},
  {'mnemonic': 'QES_GSECTOR', 'name': 'Sector'},
  {'mnemonic': 'QES_GGROUP', 'name': 'Industry Group'},
  {'mnemonic': 'QES_COUNTRY', 'name': 'Country'},
  {'mnemonic': 'CURRENCY', 'name': 'C

#### Adding a new factor to the template

In [14]:
## Add a new Factor to the template
default_risk_template.add_factor(mnemonic = 'CFOYLD_FY1', name = 'Cash Flow Yield')

#### Saving template to the database

In [15]:
## Save the template to the database. Please use 
template_name = '{}_risk_with_cash_flow'.format(user)
default_risk_template.save(template_name)

#### Ensuring the new template has the new factor we added

In [16]:
## Get Template
risk_template = connection.get_template(name = template_name, type_ = 'Risk-Model')
risk_template.json

{'__name__': 'hjain_risk_with_cash_flow',
 '__type__': 'ltool.risk.RiskModelBuilder',
 'covArgs': {'cov.period': 252,
  'expWeight': 'true',
  'interval': 3,
  'var.period': 84},
 'factors': [{'mnemonic': 'EPSYLD_LTM_B', 'name': 'Earnings Yld'},
  {'mnemonic': 'GR_INTR_EPS', 'name': 'EPS Growth (YoY)'},
  {'mnemonic': 'RTN_12M1M', 'name': 'Momentum (12M-1M)'},
  {'mnemonic': 'ES_EPS_NTM_R3M', 'name': 'Revision'},
  {'mnemonic': 'ROE', 'name': 'Profitability'},
  {'mnemonic': 'REAL_VOL', 'name': 'Volatility'},
  {'mnemonic': 'MKTCAP_M_USD', 'name': 'Size (Mkt. Cap)'},
  {'mnemonic': 'DIVYLD_TRL', 'name': 'Div Yield'},
  {'mnemonic': 'BOOKP', 'name': 'Book to Market'},
  {'mnemonic': 'CFOYLD_FY1', 'name': 'Cash Flow Yield'}],
 'meta': [{'mnemonic': 'SEDOL', 'name': 'Sedol'},
  {'mnemonic': 'TICKER', 'name': 'Ticker'},
  {'mnemonic': 'COMPANYNAME', 'name': 'Company Name'},
  {'mnemonic': 'QES_GSECTOR', 'name': 'Sector'},
  {'mnemonic': 'QES_GGROUP', 'name': 'Industry Group'},
  {'mnemonic

## Risk Model Builder

Risk Model Builder requires the following parameters to build the risk model

1. Universe: This can be one of the standard universes or custom portfolio ids
2. Template: Template containing the parameters such as factor lists, meta data list, and other parameters
3. Start Date: First date when the risk model should be built
4. End Date: Last date 
5. Frequency: How often to rebuild between the start and end date. Available frequencies are 
   1. Daily :  1d
   2. N Daily: nd
   3. Weekly: 1w
   4. Monthly: 1m
   5. Month End: 1me
   6. Quarterly: 1q
   7. Quarter End: 1qe
   8. Yearly: 1y
   9. Year End: 1ye

In [17]:
# Get a new Risk model builder
rmb = connection.get_risk_model_builder()
rmb.new_request(universe = 'SP500', template = template_name, startDate = '2020-01-31', endDate = '2022-12-31', freq = '1me')

In [18]:
## Wait until the risk model is completed. 
rmb.wait(max_wait_secs = 600)

{'dates': ['2020-01-31',
  '2020-02-29',
  '2020-03-31',
  '2020-04-30',
  '2020-05-31',
  '2020-06-30',
  '2020-07-31',
  '2020-08-31',
  '2020-09-30',
  '2020-10-31',
  '2020-11-30',
  '2020-12-31',
  '2021-01-31',
  '2021-02-28',
  '2021-03-31',
  '2021-04-30',
  '2021-05-31',
  '2021-06-30',
  '2021-07-31',
  '2021-08-31',
  '2021-09-30',
  '2021-10-31',
  '2021-11-30',
  '2021-12-31',
  '2022-01-31',
  '2022-02-28',
  '2022-03-31',
  '2022-04-30',
  '2022-05-31',
  '2022-06-30',
  '2022-07-31',
  '2022-08-31',
  '2022-09-30',
  '2022-10-31',
  '2022-11-30',
  '2022-12-31'],
 'endTime': '"2023-02-01 23:00:24.336204"',
 'message': 'Job Completed ',
 'startTime': '"2023-02-01 22:59:48.416943"',
 'status': 'SUCCESS',
 'type': 1,
 'uuid': '570abe6b-faaf-4017-a1bf-fda9449246ac'}

In [19]:
# Print out dates of the risk model that was just built
rmb.dates()

['2020-01-31',
 '2020-02-29',
 '2020-03-31',
 '2020-04-30',
 '2020-05-31',
 '2020-06-30',
 '2020-07-31',
 '2020-08-31',
 '2020-09-30',
 '2020-10-31',
 '2020-11-30',
 '2020-12-31',
 '2021-01-31',
 '2021-02-28',
 '2021-03-31',
 '2021-04-30',
 '2021-05-31',
 '2021-06-30',
 '2021-07-31',
 '2021-08-31',
 '2021-09-30',
 '2021-10-31',
 '2021-11-30',
 '2021-12-31',
 '2022-01-31',
 '2022-02-28',
 '2022-03-31',
 '2022-04-30',
 '2022-05-31',
 '2022-06-30',
 '2022-07-31',
 '2022-08-31',
 '2022-09-30',
 '2022-10-31',
 '2022-11-30',
 '2022-12-31']

### Getting the data the request. 

Risk model data can be easily accessed by query it one date at a time. The data is returned as a dictionary containing data for one day:

1. Security Exposures (*.exp)
2. Security Master (*.idm)
3. Factor Returns (over the period) (*.rtn)
4. Security Specific Risk (*.rsk)
5. Factor Mimicking Portfolio (*.fmp)
6. Security Meta (*.meta)
7. Factor Info file (*.info)



<span style="color:red">*Note that data for each risk model request is kept in the database based on UID of the request. The UID can be retrieved using the connection.jobs() function of a previous request. You can always query an old request by associating the builder with previous run UID</span>

In [20]:
data = rmb.get_data('2022-12-31')

In [21]:
## Unique Id That can be either saved by the client or it can be queries by using the connection.get_jobs()
rmb.esvc.get_id

'570abe6b-faaf-4017-a1bf-fda9449246ac'

In [22]:
connection.get_jobs()

,ENDTIME,MESSAGE,STARTTIME,STATUS,TYPEID,USER_,UUID
1,1675270141366,Job Completed,2023-02-01 16:26:17.492,SUCCESS,8,hjain,815ac724-8c39-4f04-af2b-4987176e8d82
2,1675268541268,"Error in doTryCatch(return(expr), name, parent...",2023-02-01 16:22:21.143,ERROR,1,hjain,2eff9a60-8fb0-418d-b6c3-9f775e4f531a
4,1675268426857,"Error in doTryCatch(return(expr), name, parent...",2023-02-01 16:20:26.647,ERROR,1,hjain,160050e0-2d10-4dbd-856d-ea5075e06a02
3,1675110043467,Job Completed 0,2023-01-30 20:20:26.707,SUCCESS,7,hjain,10751a63-81b9-4900-adee-a01e771273fb
0,1675107215947,Job Completed 0,2023-01-30 19:33:09.026,SUCCESS,7,hjain,f2bd799a-86ea-4070-a769-569dbaecba7c


In [23]:
print(data.keys())

dict_keys(['2022-12-31/R1_D1_20221231.rtn', '2022-12-31/R1_D1_20221231.info', '2022-12-31/R1_D1_20221231.meta', '2022-12-31/R1_D1_20221231.idm', '2022-12-31/R1_D1_20221231.exp', '2022-12-31/R1_D1_20221231.fmp', '2022-12-31/R1_D1_20221231.rsk', '2022-12-31/R1_D1_20221231.cov'])


#### Factor Covariance Matrix (*.cov)

Factor covariance matrix provides ex-ante covariance based on the risk model estimation as of that point in time. The risk of a security or a portfolio can be estimated by combining the exposure with the covariance. The covariance values are annualized. The diagonal element is the estimation of volatility of the corresponding factor. 

In [24]:
data['2022-12-31/R1_D1_20221231.cov'].head()

,Earnings Yld,EPS Growth (YoY),Momentum (12M-1M),Revision,Profitability,Volatility,Size (Mkt. Cap),Div Yield,Book to Market,Cash Flow Yield,...,Banks,Diversified Financials,Insurance,Software & Services,Technology Hardware & Equipment,Semiconductors & Semiconductor Equipment,Telecommunication Services,Media & Entertainment,Utilities,Real Estate
Earnings Yld,0.000867,-1.456918e-04,0.000084,0.000075,-6.126940e-04,-0.000020,0.000093,-0.000069,-0.000541,-0.000342,...,-0.001136,-0.000701,-0.000959,-0.000813,-0.000293,0.000387,-0.000753,-0.000709,-0.001220,-0.001202
EPS Growth (YoY),-0.000146,1.894782e-04,-0.000054,-0.000049,7.226668e-07,0.000066,-0.000005,0.000004,0.000016,0.000034,...,0.000236,0.000456,0.000337,0.000466,0.000189,0.000103,0.000341,0.000262,0.000283,0.000118
Momentum (12M-1M),0.000084,-5.378970e-05,0.002396,-0.000071,-9.733558e-05,0.000199,-0.000224,0.000377,0.000280,0.000968,...,0.001703,-0.001021,-0.001004,-0.002481,-0.000765,-0.002447,-0.002143,-0.000727,-0.003325,-0.000704
Revision,0.000075,-4.914537e-05,-0.000071,0.000335,-7.246498e-05,0.000138,-0.000025,-0.000024,-0.000008,0.000061,...,0.000312,0.000431,0.000461,0.000175,0.000452,0.000695,-0.000209,0.000317,0.000312,0.000517
Profitability,-0.000613,7.226668e-07,-0.000097,-0.000072,9.876303e-04,-0.000262,-0.000132,0.000060,0.000771,0.000141,...,0.000693,0.000036,0.000540,0.000095,-0.000046,-0.001048,0.000266,-0.000233,0.000748,0.000867


#### Factor Exposure Data (*.exp)

The file provides the factor exposure for each of the security in the risk universe. 

In [25]:
data['2022-12-31/R1_D1_20221231.exp'].head()

,Earnings Yld,EPS Growth (YoY),Momentum (12M-1M),Revision,Profitability,Volatility,Size (Mkt. Cap),Div Yield,Book to Market,Cash Flow Yield,...,Banks,Diversified Financials,Insurance,Software & Services,Technology Hardware & Equipment,Semiconductors & Semiconductor Equipment,Telecommunication Services,Media & Entertainment,Utilities,Real Estate
ID,,,,,,,,,,,,,,,,,,,,,
K9RQE6VQGZ,-4.242987,2.617901,-0.802901,2.881750,-4.112408,1.424826,-2.531873,-0.873225,-3.534483,4.741879,...,0,0,0,0,0,0,0,0,0,0
04M5J6K6G4,-0.137531,-0.633320,0.364863,1.298046,-0.985710,-1.320326,-0.461408,0.753891,1.067618,0.926281,...,0,0,0,0,0,0,0,0,1,0
04M5JN8NG4,-0.734919,-0.273393,-1.110703,-0.408698,-0.655853,-0.749443,1.012451,0.335223,-0.439371,-0.988990,...,0,0,0,0,0,0,0,0,0,0
04M5O1EOK4,-0.351049,-2.192030,-0.830495,-1.497630,0.179887,1.271188,2.134780,-1.074162,0.554642,-1.088899,...,0,0,0,0,0,1,0,0,0,0
04M5YO72K4,-1.066001,1.192518,0.296287,1.477637,-0.040513,-0.736599,1.312238,0.542991,-1.095331,-1.270883,...,0,0,0,0,0,0,0,0,0,0


#### Security Idio Risk (*.rsk)

The idio risk provides a security specific risk. The units are percentage, hence it should be divided by 100 prior to adding to adding the risks of securities

In [26]:
data['2022-12-31/R1_D1_20221231.rsk'].head()

,Specific Risk
ID,
K9RQE6VQGZ,0.259238
04M5J6K6G4,0.117219
04M5JN8NG4,0.179465
04M5O1EOK4,0.325602
04M5YO72K4,0.182538


#### Factor Returns File (*.rtn)

The factor returns provides percentage return. The returns are computed over the queried frequency. In order to compute decimal returns, the numbers should be divided by 100. 

In [27]:

data['2022-12-31/R1_D1_20221231.rtn'].head()

,Return
FactorName,
Earnings Yld,0.784769
EPS Growth (YoY),0.165004
Momentum (12M-1M),-0.394325
Revision,-0.092051
Profitability,-0.828019


#### Security Master (*.idm)

Basic security master file for each date listing reference information about the company

In [28]:
data['2022-12-31/R1_D1_20221231.idm'].head()

,SEDOL,TICKER,COMPANYNAME,QES_GSECTOR,QES_GGROUP,QES_COUNTRY,CURRENCY,IN_RISK_UNIV
ID,,,,,,,,
K9RQE6VQGZ,BCV7KT2,AAL,American Airlines Group Inc,Industrials,Transportation,US,USD,True
04M5J6K6G4,2048804,PNW,Pinnacle West Capital Corp,Utilities,Utilities,US,USD,True
04M5JN8NG4,2002305,ABT,Abbott Laboratories,Health Care,Health Care Equipment & Services,US,USD,True
04M5O1EOK4,2007849,AMD,Advanced Micro Devices Inc,Information Technology,Semiconductors & Semiconductor Equipment,US,USD,True
04M5YO72K4,2011602,APD,Air Products and Chemicals Inc.,Materials,Materials,US,USD,True


#### Factor Info File (*.info)

Provides details of each of the factor used in the risk model include the sector/country/currency when available. 

In [29]:
data['2022-12-31/R1_D1_20221231.info'].head()

,Category,FactorAlias
FactorName,,
EPSYLD_LTM_B,Style,Earnings Yld
GR_INTR_EPS,Style,EPS Growth (YoY)
RTN_12M1M,Style,Momentum (12M-1M)
ES_EPS_NTM_R3M,Style,Revision
ROE,Style,Profitability


#### Factor Mimicking Portfolio (*.fmp)

File provides compositing of FMP as constructed for that date. 

In [30]:
data['2022-12-31/R1_D1_20221231.fmp'].head()

,Earnings Yld,EPS Growth (YoY),Momentum (12M-1M),Revision,Profitability,Volatility,Size (Mkt. Cap),Div Yield,Book to Market,Cash Flow Yield,...,Banks,Diversified Financials,Insurance,Software & Services,Technology Hardware & Equipment,Semiconductors & Semiconductor Equipment,Telecommunication Services,Media & Entertainment,Utilities,Real Estate
ID,,,,,,,,,,,,,,,,,,,,,
K9RQE6VQGZ,0.013524,0.004058,-0.002986,0.002593,-0.028068,-0.000167,-0.001558,-0.000932,-0.031819,0.009153,...,0.011564,0.009084,-0.001072,0.001012,-0.000513,0.002721,0.003679,0.006717,-0.006575,-0.000027
04M5J6K6G4,0.000291,-0.003340,0.003720,0.004263,0.003899,0.002000,-0.005466,0.003944,0.004794,-0.002302,...,-0.006396,-0.002065,-0.004689,0.004303,0.000971,0.004541,-0.003005,0.001515,0.034063,-0.002444
04M5JN8NG4,0.002654,-0.001442,-0.005842,0.002302,0.000079,-0.001929,0.003877,0.001378,-0.000021,-0.000861,...,-0.004842,-0.003653,0.001829,-0.002013,-0.001863,-0.001424,-0.008357,-0.001512,-0.000466,-0.002804
04M5O1EOK4,-0.002250,-0.001172,-0.000929,-0.001237,0.001482,0.000320,0.001339,-0.002014,0.002648,0.000205,...,0.000722,-0.000974,0.000827,-0.001755,-0.000258,0.024726,0.001351,-0.003325,0.002670,0.002062
04M5YO72K4,-0.000362,-0.000876,0.000660,0.001463,-0.001806,-0.001577,0.001557,0.000350,-0.001586,-0.002332,...,0.002646,-0.000211,-0.002134,-0.002811,0.000533,0.002191,0.002506,0.002330,-0.001394,-0.000758
